# Assignment2 for RT2
## Importing the required packages

In [1]:
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, VBox, HBox, Label
from assignment_2_2023.msg import CustomStatus
from assignment_2_2023.srv import Input, Goal
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import threading
import time

## Setting up the interface

In [2]:
next_position_x = widgets.FloatText(
    value=1,
    description='X:',
    disabled=False,
    layout=Layout(width = '150px', align = "left")
)

next_position_y = widgets.FloatText(
    value=1,
    description='Y:',
    disabled=False,
    layout = Layout(width = '150px')
)

send_goal = Button(
    description='Send Goal',
    style=ButtonStyle(button_color='lightgreen')
)

cancel = Button(
    description='Cancel',
    layout=Layout(width='auto', align="center"),
    style=ButtonStyle(button_color='red')
)

output_display = widgets.Text(description='Status: ', disabled=True)

a = VBox(
    [
        HBox([
                Label("Please enter the next goal -> "),
                next_position_x, 
                next_position_y, 
                send_goal, 
                cancel
            ]),
        output_display,
        Label("Position graph:"),
    ]
)

## Declaring the functions to display data on the screen and callbacks for the ui

In [3]:
x_plot = []
y_plot = []

def plot(x_plot,y_plot):
    np_x_plot = np.array(x_plot)
    np_y_plot = np.array(y_plot)
    ax.plot(np_x_plot,np_y_plot, 'b')

            
def positionCallback(msg):
    global x_plot, y_plot 
    x_plot.append(msg.x)
    y_plot.append(msg.y)
    plot(x_plot,y_plot)    


def on_send_goal_pressed(a):
    send_input("("+str(next_position_x.value)+","+str(next_position_y.value)+")")
    ax.plot(next_position_x.value, next_position_y.value, 'go')
    
def on_cancel_pressed(b):
    goal = get_goal()
    send_input("cancel")
    ax.plot(goal.goal_x, goal.goal_y, 'ro')
    
def set_goal_status():
    counter = 0
    while(True):
        goal = get_goal()
        output_display.value = goal.status
        time.sleep(1)
    
send_goal.on_click(on_send_goal_pressed)
cancel.on_click(on_cancel_pressed)

## Declaring the subscriber and initializing the node

In [4]:
%matplotlib widget
rospy.init_node('input_sender')
sub = rospy.Subscriber('assignment_2_2023/customStatus', CustomStatus, positionCallback)

## Displaying the interface and initializing the matplotlib figure

In [5]:
# Displaying the interface
display(a)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.set_title("Assignment display")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.invert_xaxis()
ax.invert_yaxis()
ax.set_aspect('equal', adjustable='box')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Displaying the borders of the arena and the obstacles in the middle

In [6]:
# box border
ax.plot([9.5,-9.5,-9.5,9.5,9.5],[9.5,9.5,-9.5,-9.5,9.5],'y')
# obstacles
ax.plot([4, -5, -5], [-0.5, -0.5, -5], 'brown')
ax.plot([-7,5], [4.5,4.5], 'brown')

## Waiting and subscribing to the required services

In [7]:
rospy.wait_for_service('/assignment2_2023/input_string');
rospy.wait_for_service('/assignment_2_2023/last_goal');
send_input = rospy.ServiceProxy('/assignment2_2023/input_string', Input)
get_goal = rospy.ServiceProxy('/assignment_2_2023/last_goal', Goal)

## Starting the thread that periodically calls the service for the goal status

In [8]:
thread = threading.Thread(target=set_goal_status)
thread.start()